# Covid-19 Dashboard 
**A simple dashboard displaying spread of covid-19 in a world map.**
<br>
Source: github.com/CSSEGISandData/COVID-19
***

In [ ]:
import numpy as np
import pandas as pd
import folium
from folium.plugins import MarkerCluster

In [ ]:
death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
country_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')

In [ ]:
country_df.columns = map(str.lower, country_df.columns)
confirmed_df.columns = map(str.lower, confirmed_df.columns)
death_df.columns = map(str.lower, death_df.columns)
recovered_df.columns = map(str.lower, recovered_df.columns)
confirmed_df = confirmed_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
recovered_df = confirmed_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
death_df = death_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
country_df = country_df.rename(columns={'country_region': 'country'})


In [ ]:
world_map = folium.Map( zoom_start=2, tiles='CartoDB dark_matter', TileLayer = ('Stamen Terrain'))

marker_cluster = MarkerCluster().add_to(world_map)

for i in range(0,len(confirmed_df)):
    folium.Circle(
        location=[confirmed_df.iloc[i]['lat'], confirmed_df.iloc[i]['long']],
        
        
        fill=True,
        radius=(int((np.log(confirmed_df.iloc[i,-1]+1.00001)))+0.2)*25000,
        color='red',
        fill_color='blue',
        tooltip = "<div style='margin: 0; background-color: #171C26; color: #20D717;'>"+
                    "<h4 style='text-align:center;font-weight: bold'>"+confirmed_df.iloc[i]['country'] + "</h3>"
                    "<hr style='margin:10px;color: white;'>"+
                    "<ul style='color: white;list-style-type:circle;align-item:left;padding-left:200px;padding-right:200px'>"+
                        "<li>Confirmed: "+str(confirmed_df.iloc[i,-1])+"</li>"+
                        "<li>Deaths:   "+str(death_df.iloc[i,-1])+"</li>"+
                        "<li>Death Rate: "+ str(np.round(death_df.iloc[i,-1]/(confirmed_df.iloc[i,-1]+1.00001)*100,2))+ "</li>"+
                    "</ul></div>",
        ).add_to(marker_cluster)

world_map


In [ ]:
india_map = folium.Map(location=[21.1458,79.0882],zoom_start=5)
india_map.add_child(folium.GeoJson(data=(open('/Users/jayanpraveen/Downloads/web-map/india_states.json','r',encoding='utf-8-sig').read())))
marker_cluster = MarkerCluster().add_to(world_map)

plot = pd.read_csv('/Users/jayanpraveen/Desktop/data.csv')

url = "https://api.covid19india.org/csv/latest/district_wise.csv"
data = pd.read_csv(url)

LAT=list(plot['Latitude'])
LON=list(plot['longitude'])


find = list (data["District"])
data.set_index('District', inplace = True) 

for i in range(727):
    district = plot.at[i,'District']
    store = data.loc[district]
    if district in find:
        india_map.add_child(folium.CircleMarker(
            location=[LAT[i],LON[i]],
            fill=False,
            color='red',
            fill_color= 'blue',
            radius= 2.5,
            tooltip = "<div style=' background-color: #171C26; color: #20D717;'>"+
                      "<h4 style='font-size: 175%;text-align:center;font-weight: bold'>" + district + "</h3>"
                      "<hr style='margin:10px;color: white;'>"+
                      "<ul style='font-size: 125%;color: cyan;list-style-type:circle;align-item:left;padding-left:200px;padding-right:200px'>"+
                        "<li>Confirmed: "+str(store["Confirmed"])+"</li>"+
                        "<li>Active: "+str(store["Active"])+"</li>"+
                        "<li>Recovered: "+str(store["Recovered"])+"</li>"+
                        "<li>Deceased: "+str(store["Deceased"])+"</li>"+
                      "</ul></div>",
            ))

india_map